In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools
  ! pip install ipyfilechooser

In [ ]:
# import the required libraries
from czitools.metadata_tools import czi_metadata as czimd
from czitools.read_tools import read_tools as czird
from czitools.utils import misc
from ipyfilechooser import FileChooser
from IPython.display import display, HTML
from pathlib import Path
import os
import dask.array as da
import requests
import glob
import ipywidgets as widgets

if not IN_COLAB:
    from czitools.napari_tools import napari_tools
    import napari


In [ ]:
# try to find the folder with data and download otherwise from GitHub.

# Folder containing the input data
if IN_COLAB:
    INPUT_FOLDER = 'data/'
if not IN_COLAB:
    INPUT_FOLDER = '../../data/'

# Path to the data on GitHub
GITHUB_IMAGES_PATH = "https://raw.githubusercontent.com/sebi06/czitools/main/data.zip"

# Download data
if not (os.path.isdir(INPUT_FOLDER)):
    compressed_data = './data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_IMAGES_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

In [ ]:
if not IN_COLAB:
    # choose local file
    fc = FileChooser()
    fc.default_path = INPUT_FOLDER
    fc.filter_pattern = '*.czi'
    display(fc)

elif IN_COLAB:
    # list files inside the folder on gdrive
    czifiles = glob.glob(os.path.join(INPUT_FOLDER, "*.czi"))
    wd = widgets.Select(
        options=czifiles,
        description='CZI Files:',
        layout={'width': 'max-content'}
    )
    display(wd)

In [ ]:
if not IN_COLAB:
    filepath = fc.selected
elif IN_COLAB:
    filepath = wd.value

print(f"Selected File: {filepath}")

In [ ]:
# get the complete metadata at once as one big class
mdata = czimd.CziMetadata(filepath)

# get the CZI metadata dictionary directly from filename
mdict = czimd.create_md_dict_red(mdata, sort=False, remove_none=True)

# convert metadata dictionary to a pandas dataframe
mdframe = misc.md2dataframe(mdict)

# create a ipywdiget to show the dataframe with the metadata
wd = widgets.Output(layout={"scrollY": "auto", "height": "300px"})

with wd:
    display(HTML(mdframe.to_html()))
display(widgets.VBox(children=[wd]))

In [ ]:
# return array with dimension order STCZYX(A)
array6d, mdata= czird.read_6darray(filepath, use_dask=False, chunk_zyx=False)

# show dask array structure
if isinstance(array6d, da.Array):
    print(array6d)
else:
    print("Shape:", array6d.shape, "dtype:", array6d.dtype)


In [ ]:
if not IN_COLAB:

    # show array inside napari viewer
    viewer = napari.Viewer()
    layers = napari_tools.show(viewer, array6d, mdata,
                               blending="additive",
                               contrast='from_czi',
                               gamma=0.85,
                               show_metadata="tree",
                               name_sliders=True)

In [ ]:
napari.utils.nbscreenshot(viewer)